In [117]:
import random
def str_tobinary(inputstring):
    return ''.join(bin(x)[2:].zfill(8) for x in inputstring.encode('UTF-8'))

def gen_permuindex(n): #pass in n to generate a n-bit permutation index
    templist = []
    for _ in range (0,n):
        templist.append(str(_)) 
    random.shuffle(templist) #returns the permutation index
    return templist
    
def key_64to56(inputstring): #index range 0-55
    tempstring = ""
    for i in range(len(inputstring)):
        if (i+1) % 8 == 0:
            pass
        else:
            tempstring += inputstring[i]
    return tempstring

def key_56to48(inputstring): #index range 0-55
    tempstring = ""
    for i in range(len(inputstring)):
        if (i+1) % 7 == 0:
            pass
        else:
            tempstring += inputstring[i]
    return tempstring

def splithalf_l(inputstring):
    try:
        lefthalf = inputstring [:int(len(inputstring)/2)]
        return lefthalf
    except:
        print (f"input length error: length {int(len(inputstring)/2)}")

def splithalf_r(inputstring):
    try:
        righthalf = inputstring [int(len(inputstring)/2):]
        return righthalf
    except:
        print (f"input length error: length {int(len(inputstring)/2)}")

def permutation(inputstring,inputindex):
    tempstring = ""
    for _ in inputindex: #55
        tempstring += inputstring[int(_)] #permutation
    return tempstring

def permutation_r(inputstring,inputindex):
    templist = []
    x = 0
    for _ in inputstring:
        templist.append(" ")
    for _ in inputindex: #55
        templist[int(_)] = inputstring[x] #permutation
        x += 1    
    return "".join(templist)

def leftshift(inputstring,n):
    return inputstring[n:] + inputstring[0:n]

def key_gen(inputstring): 
    key_uncryp = inputstring #always 8 digitals to do: change this line!!!
    key_bi = str_tobinary(key_uncryp)
    key_bi = key_64to56(key_bi) #the 8th bit is got ridden of
    tempstring = ""
    key_bi = permutation(key_bi,PC1_index)
    subkey=[]
    stf = 0
    for i in range(0,16):
        dicttemp = str(i+1)
        keyleft = leftshift(splithalf_l(key_bi),rounddict[dicttemp])
        keyright = leftshift(splithalf_r(key_bi),rounddict[dicttemp])
        key_bi = keyleft + keyright
        key_perm = permutation(key_56to48(key_bi),PC2_index)
        subkey.append(key_perm)
    return subkey

def text_to_64bit(inputstring): #cut messeage block into a lists that contains 64bit units
    outputlist = []
    #convert inputstring into 8 digital string
    while len (inputstring) != 0:
        if 0 < len(inputstring) < 8:
            for _ in range(0,8 - len(inputstring)):
                inputstring += " "
        else:
            tempstring = inputstring[:8]
            inputstring = inputstring[8:]
            outputlist.append(str_tobinary(tempstring))
    return outputlist

def expand_indexgen(n):
    index_one = gen_permuindex(int(n*2/3)) #put index_two into index_one
    index_two = gen_permuindex(int(n*2/3)) #serve as a random array from 0 to 32
    index_new = []
    x = 0
    for i in range(len(index_one)+1):#
        if i % 4 == 0 and i != 0:
            index_new.append(index_two[x])
            index_new.extend(index_one[i-4:i]) 
            index_new.append(index_two[x+1])
    return index_new

def xor(mes,subkey):
    newstring = ""
    try:
        for _ in range(len(mes)):
            if mes[_] == subkey[_]:
                newstring += "0"
            else:
                newstring += "1"
        return (newstring)
    except:
        print ("XOR failed, check input string length")

def rand_table_gen4x16(): # to access table use array[y][x]
    line = []
    array = []
    a = 0
    b = 0
    c = 0
    while a == b or b == c:
        a = random.randint(0,15)
        b = random.randint(0,15)
        c = random.randint(0,15)
    for _ in range (0,16):
        line.append(_)
    random.shuffle(line)
    array.append(leftshift(line,0))
    array.append(leftshift(line,a))
    array.append(leftshift(line,b))
    array.append(leftshift(line,c))
    return array

def table_48to32_logic(inputstring,table):
    tempstring = ""
    output = ""
    while len (inputstring) != 0: # take 6 bits and cipher 4 bits in the middle
        tempstring = inputstring[:6]
        inputstring = inputstring[6:]
        y = int(tempstring[0] + tempstring[5],2)
        x = int(tempstring[1:5],2)
        output += '{0:04b}'.format(table[y][x])  #returns the dicimal value of the table value
    return output

def logic_64_encrypt(inputstring,keys_input):
    mes_left = splithalf_l(inputstring) #split
    mes_right = splithalf_r(inputstring) #split
   
    for i in range (0,16): #cipher loop
        mes_right_temp = permutation(mes_right,expandindex) #expand to 48 bits
        mes_right_temp = xor(mes_right_temp,keys_input[i])#
        mes_right_temp = table_48to32_logic(mes_right_temp,table) #cipher and shrink to 32 bits again
        permutation(mes_right_temp,Perm_ciph)
        mes_right = xor(mes_right_temp,mes_left)
        mes_left = mes_right
    #end of the loop  
    mes_right_temp =  mes_right
    mes_right = mes_left
    mes_left = mes_right_temp
    return permutation_r(mes_left + mes_right,Perm_init)#reverse initial permutation
    

rounddict = {"1":1,"2":1,"3":2,"4":2,"5":2,"6":2,"7":2,"8":2,"9":1,"10":2,"11":2,"12":2,"13":2,"14":2,"15":2,"16":2}

In [122]:
PC1_index = gen_permuindex(56)
PC2_index = gen_permuindex(48)
Perm_init = gen_permuindex(64)
Perm_ciph = gen_permuindex(32)
expandindex = expand_indexgen(48)
key_input = "DieWille"
subkeys = key_gen(key_input)
table = rand_table_gen4x16()
inputstring = input ("Enter any string:")
inputlist =  text_to_64bit(inputstring)
for i in range(len(inputlist)):
    outputstring += logic_64_encrypt(inputlist[i],subkeys)
print (outputstring)

Enter any string:great job!
00001100000000101000000000110001000000011010000000010110111011000101110110111010010001001111001110001111001011101110010101111000


In [ ]:
len("1111001100010010100000100101100111110011000100101000001001011001")

In [52]:
len("10101100011010110010101101011111")

32